# Artificial Neural Networks and Deep Learning

---

## Homework 2: Minimal Working Example

To make your first submission, follow these steps:
1. Create a folder named `[2024-2025] AN2DL/Homework 2` in your Google Drive.
2. Upload the `mars_for_students.npz` file to this folder.
3. Upload the Jupyter notebook `Homework 2 - Minimal Working Example.ipynb`.
4. Load and process the data.
5. Implement and train your model.
6. Submit the generated `.csv` file to Kaggle.


## 🌐 Connect Colab to Google Drive

In [1]:
from google.colab import drive

drive.mount("/gdrive")
# Change if necessary
%cd /gdrive/My Drive/Uni/Magistrale/Poli/Artificial Neural Networks and Deep Learning/challenges/challenge2

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Uni/Magistrale/Poli/Artificial Neural Networks and Deep Learning/challenges/challenge2


## ⚙️ Import Libraries

In [2]:
import os
from datetime import datetime

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tfk.__version__}")
print(f"GPU devices: {len(tf.config.list_physical_devices('GPU'))}")

TensorFlow version: 2.17.1
Keras version: 3.5.0
GPU devices: 1


## ⏳ Load the Data

In [3]:
data = np.load("mars_for_students.npz")

training_set = data["training_set"]
X_train = training_set[:, 0]
y_train = training_set[:, 1]

X_test = data["test_set"]

# Separiamo il train set in training e validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)



input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))



# Add color channel and rescale pixels between 0 and 1
X_train = X_train[..., np.newaxis] / 255.0
###
X_val = X_val[..., np.newaxis] / 255.0
X_test = X_test[..., np.newaxis] / 255.0

input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))


In [4]:
print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f"Validation X shape: {X_val.shape}")
print(f"Validation y shape: {y_val.shape}")
print(f"Test X shape: {X_test.shape}")
print(f"Input shape: {input_shape}")
print(f"Number of classes: {num_classes}")


Training X shape: (2092, 64, 128, 1)
Training y shape: (2092, 64, 128)
Validation X shape: (523, 64, 128, 1)
Validation y shape: (523, 64, 128)
Test X shape: (10022, 64, 128, 1)
Input shape: (64, 128, 1)
Number of classes: 5


## 🛠️ Train and Save the Model

In [5]:
def unet(input_shape, num_classes):

    def conv_block(x, filters):
        x = tfkl.Conv2D(filters, (3, 3), padding="same")(x)
        x = tfkl.BatchNormalization()(x)
        x = tfkl.ReLU()(x)
        x = tfkl.Conv2D(filters, (3, 3), padding="same")(x)
        x = tfkl.BatchNormalization()(x)
        x = tfkl.ReLU()(x)
        return x

    def encoder_block(x, filters):
        conv = conv_block(x, filters)
        pool = tfkl.MaxPooling2D((2, 2))(conv)
        return conv, pool

    def decoder_block(x, skip, filters):
        x = tfkl.Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding="same")(x)
        x = tfkl.Concatenate()([x, skip])
        x = conv_block(x, filters)
        return x

    inputs = tfkl.Input(input_shape)

    enc1, pool1 = encoder_block(inputs, 32)
    enc2, pool2 = encoder_block(pool1, 64)
    enc3, pool3 = encoder_block(pool2, 128)
    enc4, pool4 = encoder_block(pool3, 256)

    bottleneck = conv_block(pool4, 512)

    dec4 = decoder_block(bottleneck, enc4, 256)
    dec3 = decoder_block(dec4, enc3, 128)
    dec2 = decoder_block(dec3, enc2, 64)
    dec1 = decoder_block(dec2, enc1, 32)

    outputs = tfkl.Conv2D(num_classes, (1, 1), activation="softmax")(dec1)

    return Model(inputs, outputs)

In [6]:
model = unet(input_shape=(64, 128, 1), num_classes=5)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 64, 128, 1)     │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 64, 128, 32)    │            320 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 64, 128, 32)    │            128 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu (ReLU)              │ (None, 64, 128, 32)    │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 64, 128, 32)    │          9,248 │ re_lu[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 64, 128, 32)    │            128 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_1 (ReLU)            │ (None, 64, 128, 32)    │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 32, 64, 32)     │              0 │ re_lu_1[0][0]          │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 32, 64, 64)     │         18,496 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 32, 64, 64)     │            256 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_2 (ReLU)            │ (None, 32, 64, 64)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 32, 64, 64)     │         36,928 │ re_lu_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 32, 64, 64)     │            256 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_3 (ReLU)            │ (None, 32, 64, 64)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 16, 32, 64)     │              0 │ re_lu_3[0][0]          │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 16, 32, 128)    │         73,856 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4

 Total params: 7,771,429 (29.65 MB)

 Trainable params: 7,765,541 (29.62 MB)

 Non-trainable params: 5,888 (23.00 KB)

In [7]:
# Define the MeanIoU ignoring the background class
mean_iou = tfk.metrics.MeanIoU(num_classes=num_classes, ignore_class=0, sparse_y_pred=False)

# Define Callbacks
callbacks = [
    EarlyStopping(monitor="val_mean_io_u", patience=10, restore_best_weights=True),
    ModelCheckpoint("Tom_best_model.keras", save_best_only=True, monitor="val_mean_io_u"),
    ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-6)
]

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss="sparse_categorical_crossentropy",
              metrics=[mean_iou])

In [9]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=32,
    callbacks=callbacks
)

Epoch 1/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 62s 474ms/step - loss: 1.4810 - mean_io_u: 0.1599 - val_loss: 4.6094 - val_mean_io_u: 0.0684 - learning_rate: 0.0010
Epoch 2/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 6s 90ms/step - loss: 1.1691 - mean_io_u: 0.2257 - val_loss: 3.5371 - val_mean_io_u: 0.0684 - learning_rate: 0.0010
Epoch 3/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 6s 97ms/step - loss: 1.0675 - mean_io_u: 0.2843 - val_loss: 3.5770 - val_mean_io_u: 0.0547 - learning_rate: 0.0010
Epoch 4/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 7s 103ms/step - loss: 0.9848 - mean_io_u: 0.3212 - val_loss: 4.0916 - val_mean_io_u: 0.0546 - learning_rate: 0.0010
Epoch 5/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 10s 97ms/step - loss: 0.9299 - mean_io_u: 0.3425 - val_loss: 4.0080 - val_mean_io_u: 0.0539 - learning_rate: 0.0010
Epoch 6/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 10s 91ms/step - loss: 0.9118 - mean_io_u: 0.3443 - val_loss: 3.1004 - val_mean_io_u: 0.0591 - learning_rate: 0.0010
Epoch 7/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 10s 89ms/step - loss: 0.8743 - m

In [10]:
timestep_str = datetime.now().strftime("%y%m%d_%H%M%S")
model_filename = f"model_{timestep_str}.keras"
model.save(model_filename)
del model

print(f"Model saved to {model_filename}")

Model saved to model_241201_152111.keras


## 📊 Prepare Your Submission

In our Kaggle competition, submissions are made as `csv` files. To create a proper `csv` file, you need to flatten your predictions and include an `id` column as the first column of your dataframe. To maintain consistency between your results and our solution, please avoid shuffling the test set. The code below demonstrates how to prepare the `csv` file from your model predictions.




In [11]:
# If model_filename is not defined, load the most recent model from Google Drive
if "model_filename" not in globals() or model_filename is None:
    files = [f for f in os.listdir('.') if os.path.isfile(f) and f.startswith('model_') and f.endswith('.keras')]
    files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    if files:
        model_filename = files[0]
    else:
        raise FileNotFoundError("No model files found in the current directory.")

In [12]:
model = tfk.models.load_model(model_filename)
print(f"Model loaded from {model_filename}")

Model loaded from model_241201_152111.keras


In [13]:
preds = model.predict(X_test)
preds = np.argmax(preds, axis=-1)
print(f"Predictions shape: {preds.shape}")

314/314 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step
Predictions shape: (10022, 64, 128)


In [14]:
def y_to_df(y) -> pd.DataFrame:
    """Converts segmentation predictions into a DataFrame format for Kaggle."""
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [15]:
# Create and download the csv submission file
timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_{timestep_str}.csv"
submission_df = y_to_df(preds)
submission_df.to_csv(submission_filename, index=False)

from google.colab import files
files.download(submission_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#  
<img src="https://airlab.deib.polimi.it/wp-content/uploads/2019/07/airlab-logo-new_cropped.png" width="350">

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Instagram_logo_2022.svg/800px-Instagram_logo_2022.svg.png" width="15"> **Instagram:** https://www.instagram.com/airlab_polimi/

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/LinkedIn_icon.svg/2048px-LinkedIn_icon.svg.png" width="15"> **LinkedIn:** https://www.linkedin.com/company/airlab-polimi/
___
Credits: Alberto Archetti 📧 alberto.archetti@polito.it





```
   Copyright 2024 Alberto Archetti

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
```